<a href="https://colab.research.google.com/github/ZhongyuGuo/M4R/blob/main/Other_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import re
import numpy as np
from sklearn.linear_model import LogisticRegression
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import GridSearchCV
import sklearn

In [2]:
from google.colab import drive
drive.mount("/content/drive")
df=pd.read_csv("/content/drive/My Drive/M4R/papers2019_200000.csv")

Mounted at /content/drive


In [3]:
TRAIN_FRACTION=0.7

In [4]:
# List of categories
catlist=['math','cs','quant-ph','hep-ex','hep-th','astro-ph','nucl-ex','nucl-th']

In [5]:
# Categories to classify
cat1=catlist[6]
cat2=catlist[7]

In [6]:
# Obtain corresponding abstarcts
df1=df[df['categories']==cat1][0:1500].reset_index(drop=True)
df2=df[df['categories']==cat2][0:1500].reset_index(drop=True)
text=df1.append(df2).reset_index(drop=True)

In [7]:
def format_text(text):
    """Add spaces around punctuation and remove references to images/citations."""

    # Add spaces around punctuation
    text = re.sub(r'(?<=[^\s0-9])(?=[.,;?])', r' ', text)

    # Remove references to figures
    text = re.sub(r'\((\d+)\)', r'', text)

    text=re.sub(' the ',' ',text)
    text=re.sub(' of ',' ',text)
    text=re.sub(' and ',' ',text)
    text=re.sub(' a ',' ',text)
    text=re.sub(' in ',' ',text)
    text=re.sub(' to ',' ',text)
    text=re.sub(' with ',' ',text)
    text=re.sub(' for ',' ',text)
    text=re.sub(' by ',' ',text)
    text=re.sub(' on ',' ',text)
    text=re.sub(' as ',' ',text)
    text=re.sub(' an ',' ',text)
    text=re.sub(' at ',' ',text)
    text=re.sub(' we ',' ',text)
    text=re.sub(' is ',' ',text)
    text=re.sub(' this ',' ',text)
    text=re.sub(' are ',' ',text)
    text=re.sub(' which ',' ',text)
    text=re.sub(' be ',' ',text)
    text=re.sub(' it ',' ',text)
    text=re.sub(' that ',' ',text)
    text=re.sub(' from ',' ',text)
    text=re.sub(' can ',' ',text)
    text=re.sub(' these ',' ',text)
    text=re.sub(' our ',' ',text)
    text=re.sub(' has ',' ',text)
    text=re.sub(' have ',' ',text)
    text=re.sub('.We ','.',text)
    text=re.sub('.That ','.',text)
    text=re.sub('.The ','.',text)
    text=re.sub('.From ','.',text)
    text=re.sub('.Our ','.',text)
    text=re.sub('.In ','.',text)
    text=re.sub('.These ','.',text)
    text=re.sub('.This ','.',text)
    text=re.sub(',that ',',',text)

    # Remove double spaces
    text = re.sub(r'\s\s', ' ', text)
    
    return text

In [8]:
formatted = []

# Iterate through all the original abstracts
for a in text['abstracts']:
    formatted.append(format_text(a))

In [9]:
def make_sequences(texts,
                   training_length=50,
                   lower=True,
                   filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'):
    """Turn a set of texts into sequences of integers"""

    # Create the tokenizer object and train on texts
    tokenizer = Tokenizer(lower=lower, filters=filters)
    tokenizer.fit_on_texts(texts)

    # Create look-up dictionaries and reverse look-ups
    word_idx = tokenizer.word_index
    idx_word = tokenizer.index_word
    num_words = len(word_idx) + 1
    word_counts = tokenizer.word_counts

    print(f'There are {num_words} unique words.')

    # Convert text to sequences of integers
    features = tokenizer.texts_to_sequences(texts)

    categories=list(set(text['categories'].values))
    labels=np.zeros(len(text))
    for i in range(len(text)):
      ind=categories.index(text.iloc[i,1])
      labels[i]=ind
    return word_idx, idx_word, num_words, word_counts, features,labels


In [10]:
TRAINING_LENGTH = 50
filters = '.,!"#$%&()*+/:<=>@[\\]^_`{|}~\t\n'
word_idx, idx_word, num_words, word_counts,features,labels = make_sequences(
    formatted, TRAINING_LENGTH, lower=True, filters=filters)

There are 14260 unique words.


In [11]:
#Make all sequences to same length
pad = len(max(features, key=len))
features=[i + [0]*(pad-len(i)) for i in features]
features1=np.array(features)

In [12]:
# Define Training and Test sets
X_input=np.zeros([2000,len(features[0])])
X_input[0:1000,:]=features1[0:1000,:]
X_input[1000:2000]=features1[1500:2500,:]
X_test=np.zeros([1000,len(features[0])])
X_test[0:500,:]=features1[1000:1500,:]
X_test[500:1000]=features1[2500:3000,:]
y_input=np.zeros(2000)
y_input[0:1000]=labels[0:1000]
y_input[1000:2000]=labels[1500:2500]
y_test=np.zeros([1000])
y_test[0:500]=labels[1000:1500]
y_test[500:1000]=labels[2500:3000]

In [13]:
from sklearn.utils import shuffle


def create_train_valid(features,
                       labels,
                       train_fraction=TRAIN_FRACTION):
    """Create training and validation features and labels."""

    # Randomly shuffle features and labels
    features, labels = shuffle(features, labels, random_state=RANDOM_STATE)

    # Decide on number of samples for training
    train_end = int(train_fraction * len(labels))

    train_features = np.array(features[:train_end])
    valid_features = np.array(features[train_end:])

    train_labels = labels[:train_end]
    valid_labels = labels[train_end:]

    # Convert to arrays
    X_train, X_valid = np.array(train_features), np.array(valid_features)
    y_train, y_valid = np.array(train_labels), np.array(valid_labels)

    # Memory management
    import gc
    gc.enable()
    del features, train_features, labels, valid_features, train_labels, valid_labels
    gc.collect()

    return X_train, X_valid, y_train, y_valid

In [14]:
RANDOM_STATE = 50
X_train, X_valid, y_train, y_valid = create_train_valid(X_input, y_input)

In [15]:
#Preprecoess the training data and test features
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled=scaler.transform(X_train)
scaler1 = preprocessing.StandardScaler().fit(X_test)
X_test_scaled=scaler1.transform(X_test)

### Logistic Regression

In [16]:
# Parameters to search for the optimal ones
parameters_LR = {'C':np.linspace(0.001,0.01,19),'solver':['lbfgs','sag','liblinear','newton-cg']}

In [17]:
#GridSeach for Logistic Regression
lr=LogisticRegression(max_iter=200000)
lr_optimal=GridSearchCV(lr,parameters_LR,cv=5,n_jobs=4)
lr_optimal.fit(X_train_scaled,y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=200000, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=4,
             param_grid={'C': array([0.001 , 0.0015, 0.002 , 0.0025, 0.003 , 0.0035, 0.004 , 0.0045,
       0.005 , 0.0055, 0.006 , 0.0065, 0.007 , 0.0075, 0.008 , 0.0085,
       0.009 , 0.0095, 0.01  ]),
                         'solver': ['lbfgs', 'sag', 'liblinear', 'newton-cg']},
             pre_dispatch='2*n_jobs', refit=True, return_train_scor

In [18]:
print('Best params:',lr_optimal.best_params_)

Best params: {'C': 0.0025, 'solver': 'lbfgs'}


In [19]:
#Show accracy on train and test set
training_predict =lr_optimal.predict(X_train_scaled)
test_predict=lr_optimal.predict(X_test_scaled)
from sklearn.metrics import accuracy_score
train_acc=accuracy_score(y_train,training_predict)
test_acc=accuracy_score(y_test,test_predict)
print('Train acc: ',train_acc*100)
print('Test acc: ',test_acc*100)

Train acc:  75.5
Test acc:  72.5


## RF

In [20]:
from sklearn.ensemble import RandomForestClassifier
# Parameter to search for thr optimal ones
parameters_RF = {'n_estimators':np.linspace(10,120,12,dtype='int'),'max_depth':np.linspace(10,50,5,dtype='int'),'max_features':np.linspace(5,10,5,dtype='int')}
rfc=RandomForestClassifier()
rfc_optimal=GridSearchCV(rfc,parameters_RF,cv=5,n_jobs=8)
rfc_optimal.fit(X_train_scaled,y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [21]:
print('Hyperparameter with biggest impact:',rfc_optimal.best_params_)

Hyperparameter with biggest impact: {'max_depth': 50, 'max_features': 5, 'n_estimators': 120}


In [22]:
#Show train and test accuracy
training_predict =rfc_optimal.predict(X_train_scaled)
test_predict=rfc_optimal.predict(X_test_scaled)
train_acc=accuracy_score(y_train,training_predict)
test_acc=accuracy_score(y_test,test_predict)
print('Train acc: ',train_acc*100)
print('Test acc: ',test_acc*100)

Train acc:  100.0
Test acc:  73.3


### Supported Vector Machine

In [23]:
from sklearn.svm import SVC
parameters_SVM = {'kernel':('linear','poly','rbf'),'degree':np.linspace(2,8,7),'gamma':('scale','auto'),'C':np.linspace(0.001,0.01,10)}
svm=SVC()
svm_optimal=GridSearchCV(svm,parameters_SVM,cv=5,n_jobs=8)
svm_optimal.fit(X_train_scaled,y_train)


GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=8,
             param_grid={'C': array([0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009,
       0.01 ]),
                         'degree': array([2., 3., 4., 5., 6., 7., 8.]),
                         'gamma': ('scale', 'auto'),
                         'kernel': ('linear', 'poly', 'rbf')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [24]:
print('Hyperparameter with biggest impact:',svm_optimal.best_params_)

Hyperparameter with biggest impact: {'C': 0.001, 'degree': 2.0, 'gamma': 'scale', 'kernel': 'linear'}


In [25]:

training_predict =svm_optimal.predict(X_train_scaled)
test_predict=svm_optimal.predict(X_test_scaled)
train_acc=accuracy_score(y_train,training_predict)
test_acc=accuracy_score(y_test,test_predict)
print('Train acc: ',train_acc*100)
print('Test acc: ',test_acc*100)

Train acc:  74.57142857142857
Test acc:  71.89999999999999
